In [1]:
import torch
import numpy as np 
import pickle
from matplotlib import pyplot as plt
from policy.PytorchTraining import PytorchRunner
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import numpy as np
from simulation.odorSimulation.aquaEnv import AquaEnv
# from simulation.odorSimulation.aquaEnvGif import AquaEnv2DCropPaste as AquaEnv


In [2]:
env_arg = {
    # env_arg
    "mode": 1,
    "boundary": np.array([(-100, 100), (-100, 100)]),  # boundary of environment
    "boundary_map": np.array([30, 20]),
    "time_step": 200,  # simulation time_length
    "total_map_x": 10,  # Heatmap resolution (10 x20)
    "total_map_y": 4,
    "map_x": 1,
    "map_y": 1,

    # water_flow
    "max_water_flow_force": 7,  # water_flow_force vector
    "min_water_flow_force": 3,  # water_flow_force vector
    "water_flow_force_shape":2,
    "water_flow_force_scale":2,
    "max_random_turbulence": 1,
    "random_water_flow": True,  # random_water_flow_force

    # training
    "episode_limit": 120,
    "feature_dim": 3,

    # reward design
    'success_reward': 300,
    
    "global_boundary": torch.tensor([[0, 300], [-100, 100]]),
    # "local_boundary": torch.tensor([[-100, 100], [-100, 100]]),
    "local_boundary": torch.tensor([[-100, 100], [-100, 100]]),
    
    "global_map_boundary": torch.tensor([[0, 30], [0, 20]]),
    "local_map_boundary": torch.tensor([[0, 20], [0, 20]]),
    
    "local_map_size": (20, 20),
    "global_map_size": (30, 20),
    
    "directions": torch.tensor([[0, -1], [0, 1]]),
    "dt":5,
}

agent_arg = {
    "start_position": np.array([0, 0]),
    "dt": 8,
    "speed": 1,
    "radius": 10,

    # possible actions
    "directions": np.array([[1, 0], [0, 1], [-1, 0], [0, -1]]),   #np.array([[0, -1], [0, 1]]), # up/down action 
    "action_dim": 4,

    # initialize
    "random_start": False,
    "random_range": np.array([[0.4, 1], [0, 1]]),
    "random_direction": True,
    
    #action mode
    'action_mode': 1, #2, #0: up_down, 1:cyclone
    
    #updown range
    "updown_range" : np.array([0, 1]),

}

source_arg = {
    "position": np.array([0, 0]),
    "source_pos_shape": 8,
    "source_pos_scale":12,
    "dt": 1,
    "decay_rate": 0.97,
    "spawn_scale": 0.2,  # source produces "concentration/spawn scale" odor packets per time_step
    "size_scale": 1.05,
    "concentration_threshold": 0.3,
    
    # initialize
    "random_start": True,
    "random_range": np.array([[0, 1], [0.3, 0.7]]),
    "random_direction": True,
    "random_concentration": True,
    "min_concentration": 0.6,
    "max_concentration": 1.0,
    "burn_in": True,
}

wind_arg = {
    # Turbulence
    "energy": 0.2,
    "length_scale": 10,
    "tau_f": 5,
    "sampling_interval": 1 / 2,
}

env_arg['action_dim'] = agent_arg['action_dim']
env_arg['directions'] = torch.tensor(agent_arg['directions'])

# generate environment

env = AquaEnv(env_arg, agent_arg, source_arg, wind_arg)

30 120


In [3]:
n_example = 12000
save_interval = 2000
save_list = ["boundary", 
             "source.concentration",
             "source.position",
             "wind.water_flow_force",
             "agent.position"
             
             "detect_total_concentration",
             "detect_avg_concentration",
             "detect_water_flow_force",
             "detect_source",
             ]

save_list = [("self", "boundary"), 
             ("self.source", "concentration"),
             ("self.source", "position"),
             ("self.wind", "water_flow_force"),
             ("self", "agent_position"),
             ("self", "agent_start"),
             ("self", "action"),
             
             ("self", "detect_total_concentration"),
             ("self", "detect_water_flow_force"),
             ("self", "detect_source"),
             
             ]
episode_data = env.getEpisodeData(save_list)

In [4]:
# file = "stochastic"
file = "cyclone"

agent_arg['action_mode'] = 1
# generate data
from tqdm import tqdm
from common.utils import Logger
data_log = Logger()
log_path = "../output/data/"+file+"/data_{}.txt"
for i_example in tqdm(range(n_example)):
    env.reset()
    episode_data = env.getEpisodeData(save_list)
    while np.sum(np.array(episode_data['detect_total_concentration'])>0)<5:
        episode_data = env.getEpisodeData(save_list)    
    data_log.addListItem(episode_data)
    
    if (i_example+1) % save_interval == 0 or i_example == 0:
        data_log.writeToFile(log_path.format(round(i_example/save_interval)))
        data_log = Logger()
        
# data_log.writeToFile(log_path.format(i_example//save_interval))

100%|██████████| 12000/12000 [9:29:07<00:00,  2.85s/it]  


In [5]:

n_example = 10000
save_interval = 2000
# generate data
from tqdm import tqdm
from common.utils import Logger
data_log = Logger()
log_path = "../output/data/"+file+"/data_{}.txt"
for i_example in tqdm(range(n_example)):
    env.reset()
    episode_data = env.getEpisodeData(save_list)
    
    data_log.addListItem(episode_data)
    
    if (i_example+1) % save_interval == 0 or i_example == 0:
        data_log.writeToFile(log_path.format(round(i_example/save_interval)))
        data_log = Logger()
        
# data_log.writeToFile(log_path.format(i_example//save_interval))

100%|██████████| 10000/10000 [4:03:10<00:00,  1.46s/it] 


In [5]:
agent_arg['action_mode'] = 1
# generate data
from tqdm import tqdm
from common.utils import Logger
data_log = Logger()
log_path = "../output/data/cyclone/data_{}.txt"
for i_example in tqdm(range(n_example)):
    env.reset()
    episode_data = env.getEpisodeData(save_list)
    # while np.sum(np.array(episode_data['detect_total_concentration'])>0)==0:
    #     episode_data = env.getEpisodeData(save_list)    
    data_log.addListItem(episode_data)
    
    if (i_example+1) % save_interval == 0 or i_example == 0:
        data_log.writeToFile(log_path.format(round(i_example/save_interval)))
        data_log = Logger()
        
data_log.writeToFile(log_path.format(i_example//save_interval))

100%|██████████| 12000/12000 [4:52:12<00:00,  1.46s/it]  


In [12]:
env.wind.getTurbulenceVector([[100, 0]], 7)

array([[ 4.65308074, -1.88963978]])

In [6]:
env.action

0

In [7]:
def getSensingObservation(self):
    def getMapPosition(positions):
        interval_map = (np.sum(np.abs(self.boundary), axis=1) / self.boundary_map)[np.newaxis, ...]  # (1, 2)
        map_positions = (positions - self.boundary[:, 0][np.newaxis, ...]) / interval_map
        return map_positions.astype(np.int)

    def updateEnvMap():
        packets_position, packets_concentration = self.source.packets_position, self.source.packets_concentration
        packets_map_position = getMapPosition(packets_position)

        map_x, map_y = self.boundary_map
        for i_x in range(map_x):
            for i_y in range(map_y):
                cell_position_mask = (packets_map_position[:, 0] == i_x) & (packets_map_position[:, 1] == i_y)
                cell_total_concentration = np.sum(packets_concentration[cell_position_mask])
                cell_n_packet = len(packets_concentration[cell_position_mask])
                self.total_concentration_map[i_x, i_y][self.t_step] = cell_total_concentration
                self.occurence_rate_map[i_x, i_y][self.t_step] = cell_n_packet

    updateEnvMap()
    agent_position = getMapPosition(self.agent.position)[0]  # (2)
    total_concen = self.total_concentration_map[agent_position[0], agent_position[1]][:self.t_step+1]  # (1,time_len)
    if len(total_concen) == 0:
        total_concen = 0
    total_concen = np.atleast_2d(np.mean(total_concen))
    # print(total_concen, np.sum(self.total_concentration_map))

    # observation = (total_concentration, water_flow_force)
    agent_water_flow_force = self.wind.getTurbulenceVector(self.agent.position[np.newaxis, ...], self.t)  # (1,2)
    obs = np.concatenate((total_concen, agent_water_flow_force), axis=-1)  # (1,3)

    # generate data purpose
    self.detect_total_concentration = total_concen
    self.detect_water_flow_force = agent_water_flow_force

    torch_obs = torch.from_numpy(obs).type(torch.float)

    # generate data purpose
    self.detect_total_concentration = total_concen
    self.detect_water_flow_force = agent_water_flow_force
    if total_concen > 0: self.detect_source = 1

    self.source_concentration = self.source.concentration
    return torch_obs

AttributeError: 'list' object has no attribute 'shape'

In [6]:
episode_data['agent_position']

[[155, 0],
 [155, 5],
 [155, 10],
 [150, 10],
 [145, 10],
 [145, 5],
 [145, 0],
 [145, -5],
 [150, -5],
 [155, -5],
 [160, -5],
 [160, 0],
 [160, 5],
 [160, 10],
 [160, 15],
 [155, 15],
 [150, 15],
 [145, 15],
 [140, 15],
 [140, 10],
 [140, 5],
 [140, 0],
 [140, -5],
 [140, -10],
 [145, -10],
 [150, -10],
 [155, -10],
 [160, -10],
 [165, -10],
 [165, -5],
 [165, 0],
 [165, 5],
 [165, 10],
 [165, 15],
 [165, 20],
 [160, 20],
 [155, 20],
 [150, 20],
 [145, 20],
 [140, 20],
 [135, 20],
 [135, 15],
 [135, 10],
 [135, 5],
 [135, 0],
 [135, -5],
 [135, -10],
 [135, -15],
 [140, -15],
 [145, -15],
 [150, -15],
 [155, -15],
 [160, -15],
 [165, -15],
 [170, -15],
 [170, -10],
 [170, -5],
 [170, 0],
 [170, 5],
 [170, 10],
 [170, 15],
 [170, 20],
 [170, 25],
 [165, 25],
 [160, 25],
 [155, 25],
 [150, 25],
 [145, 25],
 [140, 25],
 [135, 25],
 [130, 25],
 [130, 20],
 [130, 15],
 [130, 10],
 [130, 5],
 [130, 0],
 [130, -5],
 [130, -10],
 [130, -15],
 [130, -20]]

In [24]:
import torch
position = torch.tensor([0,0])
directions = torch.tensor([[0, 1], [-1, 0], [0, -1], [1, 0]])
action = 2
scale = 0.5

In [25]:
position = position + directions[action] * scale
print(position)
map_action = position // 1
position = position - position //1
print(map_action, position)

tensor([ 0.0000, -0.5000])
tensor([0., -0.]) tensor([ 0.0000, -0.5000])


In [ ]:
self.position = self.position + self.directions[action.squeeze(1)] * self.scale
        self.map_action = self.position // 1
        self.position = self.position - self.position //1

In [27]:
a = torch.tensor([-1.5])
a//1

tensor([-1.])